In [ ]:
from langchain.embeddings import LlamaCppEmbeddings

import evaluate 
import pandas as pd
from tqdm import tqdm
import torch 
from sentence_transformers import util

nDCG_metric = evaluate.load('JP-SystemsX/nDCG')
original = pd.read_csv('flights.csv')
original = original[['searchTerms', 'rank', 'snippet']]



In [ ]:
def calc_ndcg(model):

    dataset_ranks = []
    dataset_preds = []

    docs = []
    ranks = []
    for i, row in tqdm(original.iterrows()):
        docs.append(row['snippet'])
        ranks.append(row['rank'])

        if i!=0 and i%10==9:
            query = row['searchTerms']
            query_emb = torch.as_tensor(model.embed_query(query))
            docs_emb = torch.as_tensor(model.embed_documents(docs))
            cos_scores = util.cos_sim(query_emb, docs_emb)[0].cpu()
            

            results = torch.topk(cos_scores, k = 10, sorted=True)
            rank_results = results[1]+1


            dataset_ranks.append(ranks)
            dataset_preds.append(rank_results)

            ranks = []
            docs = []
        

    return nDCG_metric.compute(references = dataset_ranks, predictions = dataset_preds)

In [ ]:
with open('/home/karineayrs/jupyter_notebooks/diploma/output/llama_ndcg.txt', 'w') as f:
    llama = LlamaCppEmbeddings(model_path='./gpt4all-lora-quantized-new.bin')
    ndcg_llama = calc_ndcg(llama)
    print(ndcg_llama)
    f.write(str(calc_ndcg(llama)))
    f.write('\n')
    